## Unit Test for Phishing Detection Model

In [1]:
import numpy as np
import pandas as pd

%load_ext spl2_kernel

## Extract first 10 records from the test dataset as unit test data

In [2]:
df = pd.read_json('s3://smle-experiments/datasets/phishing_email/splunk_test.json', lines=True)[0:10]
t = [i for i in range(10)]
df['_time'] = t
df.to_json('./detect_phishing_content.json', orient='records', lines=True)

/opt/conda/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname BST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/opt/conda/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/opt/conda/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


## SPL2 string to perform model inference for phishing detection

In [3]:
%%spl2
| from read_json("s3://smle-experiments/datasets/phishing_email/detect_phishing_content.json")
| eval eventLine=concat(From, " ", Subject, " ", Content, " ", "                                                                                                                                ")
| where eventLine IS NOT NULL
| eval mapC = {" ":32,"!":33,"\"":34,"#":35,"$$":36,"%":37,"&":38,"'":39,"(":40,")":41,"*":42,"+":43,",":44,"-":45,".":46,"/":47,"0":48,"1":49,"2":50,"3":51,"4":52,"5":53,"6":54,"7":55,"8":56,"9":57,":":58,";":59,"<":60,"=":61,">":62,"?":63,"@":64,"A":65,"B":66,"C":67,"D":68,"E":69,"F":70,"G":71,"H":72,"I":73,"J":74,"K":75,"L":76,"M":77,"N":78,"O":79,"P":80,"Q":81,"R":82,"S":83,"T":84,"U":85,"V":86,"W":87,"X":88,"Y":89,"Z":90,"[":91,"\\":92,"]":93,"^":94,"_":95,"`":96,"a":97,"b":98,"c":99,"d":100,"e":101,"f":102,"g":103,"h":104,"i":105,"j":106,"k":107,"l":108,"m":109,"n":110,"o":111,"p":112,"q":113,"r":114,"s":115,"t":116,"u":117,"v":118,"w":119,"x":120,"y":121,"z":122,"{":123,"|":124,"}":125,"~":126}
| eval ml_in = for_each(
        iterator(mvrange(1,129), "i"),
        cast(map_get(mapC, substr(eventLine, i, 1)), "float") )
| apply_model connection_id="" path="s3://smle-experiments/models/xlin/phishing_email" name="phishing_email_v8" 
| eval probability = mvindex(ml_out, 0) 
| where probability > 0.5
| eval start_time = _time, end_time = _time, entities = "TBD", body = "TBD"
| select probability, body, entities, start_time, end_time
;

 Finished.                     

,end_time,start_time,body,entities,probability
0,4,4,TBD,TBD,0.999498
